*Marcello Victorino* <br>
*04/30/2019* -  

# Introduction
This project is part of a requirement to graduate in the Udacity's Data Analyst Nanodegree (*DAND*).

It provides the opportunity to implement Data Wrangling in practice by gathering data from different sources, assessing it for quality and tidiness issues and then promote the necessary cleaning task - programatically.

Finally, once the data is properly cleaned and stored, a brief analysis is conducted with visualizations, highlighting interesting insights.

The data for this project was provided in partnership with the **WeRateDogs** channel from twitter, containing over 5,000 observations about dogs.

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

## Gather

In [15]:
# Gathering Twitter Enhanced Archive data
df_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
df_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [50]:
# Gathering Image Predictions for Dog Breed - Available online
import requests
import os

# Avoid redownloading if file already saved locally
if 'dog_breed.txt' in os.listdir():
    pass

else:
    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

    # Accessing file online
    r = requests.get(url)

    # Saving content locally
    with open('dog_breed.txt', 'wb') as fh:
        fh.write(r.content)

# Reading file as Dataframe
df_breed = pd.read_csv('dog_breed.txt', sep='\t')
df_breed.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [45]:
# Gathering data from Twitter API
from time import time as timer
import tweepy
import twitter_secret_credentials as credentials

consumer_key = credentials.ckey
consumer_secret = credentials.csecret
access_token = credentials.atoken
access_secret = credentials.atoken

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [48]:
# df_archive.tweet_id.nunique() # 2,356 unique ids
tweet_ids = df_archive.tweet_id.values
# len(tweet_ids) # 2,356 OK

api.get_status(tweet_ids[0])
# Note: use try/except 
# api.get_status(tweet_id, tweet_mode='extended')

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]